In [57]:

import requests
url = 'https://arxiv.org/pdf/1706.03762'
 
 
response = requests.get(url)
file_Path = 'pdfs/research_Paper.pdf'
 
if response.status_code == 200:
    with open(file_Path, 'wb') as file:
        file.write(response.content)
    print('File downloaded successfully')
else:
    print('Failed to download file')

File downloaded successfully


In [58]:
import warnings
warnings.filterwarnings('ignore')

In [59]:
from langchain_google_vertexai import ChatVertexAI
model = ChatVertexAI(model='gemini-1.5-pro')

In [60]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [61]:
from crewai import Agent, Task, Crew, Process

In [62]:
from crewai_tools import PDFSearchTool
pdf_tool = PDFSearchTool(
    pdf='pdfs/research_Paper.pdf',
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini-1.5-pro",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

Inserting batches in chromadb:   0%|          | 0/1 [00:01<?, ?it/s]


In [63]:
Pdf_researcher = Agent(
    role="PDF Researcher",
    goal="Research on factually accurate content on {topic} from PDF",
    backstory="You're working on researching a pdf"
              "about the topic: {topic}."
              "You collect information about the features, advancements" 
              "over previous generations and details regarding architecture"
              "as well as performance of the {topic}."
              "You should only write things mentioned in the PDF."
              "Your work is the basis for "
              "the Content Planner to plan an article on this topic.",
    allow_delegation=False,
    tools=[pdf_tool],
    verbose=True,
    llm=model
)

In [64]:
Researcher = Agent(
    role="Content Researcher",
    goal="Research on factually accurate content on {topic} from web search",
    backstory="You're working on researching a blog article "
              "about the topic: {topic}."
              "You collect information about the features, advancements" 
              "over previous generations and how it will help a"
              "tech company which works on cloud and generative ai create new innovations"
              "or how it can use it to optimize their current workflow"
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Planner to plan an article on this topic.",
    allow_delegation=False,
    tools=[search_tool],
	verbose=True,
    llm=model
)

In [65]:
planner = Agent(
    role="Content Planner",
    goal="Take information from PDF Researcher and Content Researcher and plan engaging and factually accurate content",
    backstory="You're an expert blogger working at Medium and towards Data science"
              "You are working on writing blog about the topic: {topic}."
              "You collect information from both PDF Researcher and Content Researcher"
              "and plan the overall outline of the blog." 
              "consider the usefulness of information from PDF Researcher and Content Researcher"
              "Make sure to explain the architecture properly if is provided in the context"
              "and plan engaging content based on target audience."
              "Your work is the basis for "
              "the Content writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=model

)

In [66]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Researcher. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=model
)

In [67]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=model
)

In [68]:
research = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Develop a detailed content outline including "
            "an introduction, key points, features, advancements, integrations in tech company and a possible innovations.\n"
        "3. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive research on {topic} over the web"
        "with an outline, audience analysis, features, usage,"
        "SEO keywords, and resources.",
    agent=Researcher,
)

In [69]:
pdf_research = Task(
    description=(
        "1. Prioritize the architecture, features, performance,"
            "and why {topic} was introduced.\n"
        "2. Develop a detailed content outline including "
            "an introduction, key points, features, architecture, advancements.\n"
        "3. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content summarization of the pdf document "
        "with an outline, explained architecture and pros and cons with"
        "SEO keywords, and resources.",
    agent=Pdf_researcher,
)

In [70]:
plan = Task(
    description=(
        "1. Prioritize the PDF content over internet search\n"
        "2. Plan engaging content and identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, features, architecture, advancements.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [71]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [72]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [73]:
crew = Crew(
    agents=[Pdf_researcher, Researcher, planner, writer, editor],
    tasks=[pdf_research, research, plan, write, edit],
    process= Process.sequential,
    verbose=2
)

2024-07-04 19:16:34,059 - 948 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [74]:
result = crew.kickoff(inputs={"topic": "Transformers in natural language processing"})

 [2024-07-04 19:16:34][DEBUG]: == Working Agent: PDF Researcher
 [2024-07-04 19:16:34][INFO]: == Starting Task: 1. Prioritize the architecture, features, performance,and why Transformers in natural language processing was introduced.
2. Develop a detailed content outline including an introduction, key points, features, architecture, advancements.
3. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I need to find information about the architecture, features, performance, and the reasons for introducing Transformers in natural language processing. 
Action: Search a PDF's content
Action Input: {"query": "What is the architecture of Transformers in natural language processing? What are its key features and advantages over previous approaches? How does it perform on common NLP tasks? Why was it introduced?"}  

I encountered an error while trying to use the tool. This was the error: 400 API key not valid. Please pass a valid API key. [r

In [76]:
from IPython.display import Markdown
Markdown(result)

## Transformers: Reshaping the Landscape of Cloud Computing and Generative AI

**The amount of data generated globally is exploding, exceeding 90 zettabytes in 2023 alone.** For tech companies, this data deluge presents both a challenge and an opportunity.  Harnessing data insights is crucial for driving innovation and growth, yet traditional methods struggle to keep pace with the sheer volume and complexity of information. Enter Transformers, a revolutionary technology rapidly changing the landscape of Natural Language Processing (NLP) and, by extension, the way businesses operate.

**The Rise of a Game-Changer**

Traditional NLP models often falter when faced with the nuances of human language and the vastness of real-world datasets. Transformers, first introduced in 2017, have emerged as a solution, consistently outperforming previous models in accuracy and efficiency. Their impact has been nothing short of transformative, particularly in areas like cloud computing and generative AI, where they are essential for maintaining a competitive edge.

## Understanding the Power of Transformers: A New Era of NLP

**What sets Transformers apart?** Unlike their predecessors, Transformers don't process text sequentially. Instead, they leverage a mechanism called "self-attention" to analyze an entire sentence or document simultaneously, grasping relationships between words regardless of their distance. Imagine reading a book and instantly understanding how each sentence connects to the larger narrative—that's the power of self-attention.

This revolutionary approach, often visualized through diagrams showing how words "attend" to each other, offers several key advantages:

* **Parallel Processing:** Transformers can be trained and executed significantly faster than traditional models, which is crucial for handling the massive datasets common in cloud-based applications.
* **Long-Range Dependencies:** They excel at understanding connections between words even when they are far apart, leading to more accurate and nuanced language processing.
* **Contextualized Embeddings:** Transformers generate word representations that dynamically adapt to the context, enabling a deeper understanding of meaning and intent.

## Transformers in Action: Revolutionizing Cloud Computing

The real-world applications of Transformers are already reshaping cloud computing across various domains.

**Enhanced Cloud Services:**

* **Real-time Translation:**  Services like Google Translate have achieved unprecedented accuracy and fluency thanks to Transformers, breaking down language barriers for global businesses.
* **Next-Generation Chatbots and Virtual Assistants:** Transformers power more intelligent and human-like conversational AI, improving customer service and personalizing interactions. Companies like Ada and Drift are at the forefront of this transformation.
* **Hyper-Personalized Content Recommendations:** By analyzing user data, Transformers enable streaming platforms like Netflix to deliver highly targeted content suggestions, driving engagement and satisfaction.

**Revolutionizing Data Analysis and Insights:**

* **Sentiment Analysis:**  Businesses can now gauge public opinion, analyze customer feedback, and make data-driven decisions by leveraging Transformers to understand sentiment in textual data. Tools like MonkeyLearn and Brand24 are at the forefront of this revolution.
* **Document Summarization:** Extracting key insights from lengthy reports and research papers is now faster and more efficient thanks to Transformer-based summarization tools like Scholarcy and Semantic Scholar.

## Transformers: Unleashing the Potential of Generative AI

Beyond enhancing existing applications, Transformers are fueling a new wave of innovation in generative AI, pushing the boundaries of what's possible. 

**Content Creation at Scale:**

* The days of tedious content creation are fading. Transformers now power tools like Copy.ai and Jasper, enabling businesses to generate high-quality articles, marketing copy, and social media posts at scale. This frees up human writers to focus on strategy and creativity.
* Language models like GPT-3 and Jurassic-1, built on Transformer architecture, offer unprecedented fluency and coherence, further blurring the lines between human and machine-generated content.

**Code Generation and Software Development:**

* Imagine describing a software function in plain English and having an AI assistant generate the code for you. Tools like GitHub Copilot, powered by Transformers, are making this a reality, assisting developers with code completion, bug detection, and even translating code between different programming languages.
* This has the potential to significantly increase developer productivity and address the growing demand for skilled software engineers.

**Beyond Text:**

The influence of Transformers extends beyond NLP. Researchers are adapting the technology for use in areas like image recognition (Vision Transformers, used in platforms like Google Photos) and audio processing, demonstrating its versatility and vast potential. 

## Optimizing Workflow and Driving Innovation with Transformers

The transformative impact of Transformers on business operations and innovation cannot be overstated.

**Workflow Optimization:**

* **Automating Content Moderation:** Social media platforms like Facebook use Transformers to automatically identify and filter out harmful content, creating safer online spaces.
* **Streamlining Customer Support:** By powering intelligent chatbots and automating ticket routing, Transformers enhance customer service operations, leading to faster resolution times and increased customer satisfaction. 
* **Accelerated Research and Development:** Researchers can leverage Transformers to analyze vast amounts of scientific literature and patents, identifying trends and extracting insights that accelerate the pace of innovation.

**Innovation Opportunities:**

* **New Product Development:** The possibilities are endless. Imagine AI-powered writing assistants that adapt to your style, advanced language learning tools that personalize lessons, or education platforms that provide customized feedback.
* **Data-Driven Decision Making:** Sentiment analysis, trend prediction, and other NLP tasks, all enhanced by Transformers, provide valuable insights to inform critical business decisions, from product development to marketing campaigns. 

## Transformers and SEO: The Future of Content Optimization

As search engines become more sophisticated, so too must our approach to SEO. Transformers are poised to become indispensable tools for content optimization.

* **Understanding Search Intent:** Transformers can analyze search queries beyond literal keywords, deciphering the user's true intent and enabling content creators to target the most relevant audience.
* **Content Strategy and Outline Generation:** Imagine receiving AI-powered suggestions for relevant topics, headings, and subheadings based on top-performing content. Transformers can help structure content for optimal SEO performance. 
* **SEO-optimized Content Writing:** By analyzing ranking factors and user engagement metrics, Transformers can guide content creation, ensuring that it's informative, engaging, and aligned with search engine algorithms. 
* **Voice Search and Conversational SEO:** As voice search gains prominence, optimizing for natural language queries is paramount. Transformers, with their ability to understand nuances in spoken language, are essential for this evolution.

## Embracing the Transformer Revolution

The convergence of Transformers, cloud computing, and generative AI presents a unique opportunity for tech companies. By embracing this revolution, businesses can:

* **Drive Innovation:** Develop groundbreaking products and services that leverage the power of NLP.
* **Optimize Workflows:** Automate tasks, enhance efficiency, and free up human capital for strategic initiatives.
* **Unlock New Possibilities:** Gain a competitive edge by harnessing the power of data-driven insights and AI-powered solutions. 

The future belongs to those who embrace change. Explore the resources available, experiment with pre-trained Transformer models like those on Hugging Face, and start building the next generation of intelligent, cloud-based applications. The transformative journey begins now.